In [ ]:
# ------------------------------------------------------------------------
#
# TITLE - 8_assess_rotating_anisotropy
# AUTHOR - James Lane
# PROJECT - tng-dfs
#
# ------------------------------------------------------------------------
#
# Docstrings and metadata:
'''Examine the impact of the rotating perscription on the anisotropy parameter
'''

__author__ = "James Lane" 

In [ ]:
# %load ../../src/nb_modules/nb_imports.txt
### Imports

## Basic
import numpy as np
import sys, os
import dill as pickle
import time, multiprocessing, logging

## Plotting
import matplotlib as mpl
from matplotlib import pyplot as plt
import corner

## Astropy
from astropy import units as apu

## Analysis
import scipy.optimize
import scipy.stats
import scipy.interpolate
import emcee

## galpy
from galpy import orbit

## Project-specific
src_path = 'src/'
while True:
    if os.path.exists(src_path): break
    if os.path.realpath(src_path).split('/')[-1] in ['tng-dfs','/']:
            raise FileNotFoundError('Failed to find src/ directory.')
    src_path = os.path.join('..',src_path)
sys.path.insert(0,src_path)
from tng_dfs import kinematics as pkin
from tng_dfs import util as putil

### Notebook setup

%matplotlib inline
plt.style.use(os.path.join(src_path,'mpl/project.mplstyle')) # This must be exactly here
%config InlineBackend.figure_format = 'retina'
%load_ext autoreload
%autoreload 2

### Keywords, loading, pathing

In [ ]:
# %load ../../src/nb_modules/nb_setup.txt
# Keywords
cdict = putil.load_config_to_dict()
keywords = ['DATA_DIR','MW_ANALOG_DIR','FIG_DIR_BASE','FITTING_DIR_BASE',
            'RO','VO','ZO','LITTLE_H','MW_MASS_RANGE']
data_dir,mw_analog_dir,fig_dir_base,fitting_dir_base,ro,vo,zo,h,\
    mw_mass_range = putil.parse_config_dict(cdict,keywords)

# MW Analog 
mwsubs,mwsubs_vars = putil.prepare_mwsubs(mw_analog_dir,h=h,
    mw_mass_range=mw_mass_range,return_vars=True,force_mwsubs=False,
    bulge_disk_fraction_cuts=True)

# Figure path
local_fig_dir = './fig/'
fig_dir = os.path.join(fig_dir_base, 
    'notebooks/5_compare_distribution_functions/8_assess_rotating_anisotropy')
os.makedirs(local_fig_dir,exist_ok=True)
os.makedirs(fig_dir,exist_ok=True)
show_plots = False

# Load tree data
tree_primary_filename = os.path.join(mw_analog_dir,
    'major_mergers/tree_primaries.pkl')
with open(tree_primary_filename,'rb') as handle: 
    tree_primaries = pickle.load(handle)
tree_major_mergers_filename = os.path.join(mw_analog_dir,
    'major_mergers/tree_major_mergers.pkl')
with open(tree_major_mergers_filename,'rb') as handle:
    tree_major_mergers = pickle.load(handle)
n_mw = len(tree_primaries)

### Define the fitting procedure, same as was done to originally fit the anisotropy

In [ ]:
def mloglike_beta(*args, **kwargs):
    return -loglike_beta(*args, **kwargs)

def loglike_beta(params, model, r, beta, sigma=None, mass=None, 
    usr_log_prior=None, usr_log_prior_params=[], parts=False):
    # Evaluate the domain prior
    if not domain_prior_beta(model, params):
        return -np.inf
    # Evaluate the prior on the beta model
    logprior = logprior_beta(model, params)
    # Evaluate any user supplied prior
    if callable(usr_log_prior):
        usrlogprior = usr_log_prior(params, *usr_log_prior_params)
        if np.isinf(usrlogprior):
            return -np.inf
    else:
        usrlogprior = 0
    # Evaluate the model
    beta_model = model(r, *params)
    # Sigma for the likelihood
    if sigma is not None:
        _sigma = sigma
    elif mass is not None:
        mass_frac = mass/np.sum(mass)
        _sigma = 1/mass_frac
    else:
        _sigma = 0.1
    # Compute the log objective
    logobj = -0.5*np.square(beta - beta_model)/_sigma**2
    # Compute the log likelihood
    loglike = np.sum(logobj) + logprior + usrlogprior
    if parts:
        return loglike, np.sum(logobj), logprior, usrlogprior
    else:
        return loglike

def logprior_beta(model, params):
    if model.__name__ == 'beta_osipkov_merritt':
        ra, = params
        # log prior on ra
        ra_min = 0.0001
        ra_max = 10000.
        prior_ra = scipy.stats.loguniform.pdf(ra, ra_min, ra_max)
        return np.log(prior_ra)
    if model.__name__ == 'beta_cuddeford91':
        ra, alpha = params
        # log prior on ra
        ra_min = 0.0001
        ra_max = 10000.
        prior_ra = scipy.stats.loguniform.pdf(ra, ra_min, ra_max)
        return np.log(prior_ra)
    return 0.

def domain_prior_beta(model, params):
    if model.__name__ == 'beta_constant':
        beta, = params
        if beta >= 1.:
            return False
    elif model.__name__ == 'beta_osipkov_merritt':
        ra, = params
        if ra <= 0.:
            return False
    elif model.__name__ == 'beta_cuddeford91':
        ra, alpha = params
        if ra <= 0.:
            return False
        if alpha <= -1.:
            return False
    return True

### Compute the anisotropies of the DF samples 

In [ ]:
verbose = True

# Pathing
dens_fitting_dir = os.path.join(fitting_dir_base,'density_profile')
df_fitting_dir = os.path.join(fitting_dir_base,'distribution_function')
analysis_version = 'v1.1'
analysis_dir = os.path.join(mw_analog_dir,'analysis',analysis_version)

# Anisotropy profile calculation keywords
n_bs = 50

# Get the sample orbits
sample_data_cb = np.load(os.path.join(analysis_dir,'sample_data_cb.npy'),
    allow_pickle=True)
sample_data_om = np.load(os.path.join(analysis_dir,'sample_data_om.npy'),
    allow_pickle=True)

samp_anisotropy_cb = []
samp_anisotropy_om = []

for i in range(n_mw):
    # if i != 0: continue

    # Get the primary
    primary = tree_primaries[i]
    z0_sid = primary.subfind_id[0]
    major_mergers = primary.tree_major_mergers
    n_major = primary.n_major_mergers
    # n_snap = len(primary.snapnum)
    # primary_filename = primary.get_cutout_filename(mw_analog_dir,
    #     snapnum=primary.snapnum[0])
    # co = pcutout.TNGCutout(primary_filename)
    # co.center_and_rectify()
    # pid = co.get_property('stars','ParticleIDs')

    for j in range(n_major):
        # if j > 0: continue
        if verbose: print(f'Calculating anisotropy profiles for MW analog '
                          f'{i+1}/{n_mw}, merger {j+1}/{n_major}', end='\r')

        # # Get the major merger
        major_merger = primary.tree_major_mergers[j]
        major_acc_sid = major_merger.subfind_id[0]
        major_mlpid = major_merger.secondary_mlpid
        r_softening = putil.get_softening_length('stars', z=0, physical=True)

        ### Constant anisotropy samples

        # Get the sample
        mask = (sample_data_cb['z0_sid'] == z0_sid) &\
               (sample_data_cb['major_acc_sid'] == major_acc_sid) &\
               (sample_data_cb['major_mlpid'] == major_mlpid) &\
               (sample_data_cb['merger_number'] == j+1)
        indx = np.where(mask)[0]
        assert len(indx) == 1, 'Something went wrong'
        indx = indx[0]

        sample = sample_data_cb['sample'][indx]

        # Adaptive binning
        adaptive_binning_kwargs = {'rmin':np.max([np.min(sample.r().to_value(apu.kpc)), r_softening]),
                                   'n':np.min([500, len(sample)//10]),
                                   'rmax':np.max(sample.r().to_value(apu.kpc)),
                                   'bin_mode':'exact numbers',
                                   'bin_equal_n':True,
                                   'end_mode':'ignore',
                                   'bin_cents_mode':'median'}
        bin_edges, bin_cents, _ = pkin.get_radius_binning(sample, 
            **adaptive_binning_kwargs)
        bin_size = bin_edges[1:] - bin_edges[:-1]

        # Compute anisotropy profile
        compute_betas_kwargs = {'use_dispersions':True,
                               'return_kinematics':True}
        beta,vr2,vp2,vt2 = \
            pkin.compute_betas_bootstrap(sample, bin_edges, n_bootstrap=n_bs,
                compute_betas_kwargs=compute_betas_kwargs)

        # Store data
        _data = (
            z0_sid,
            major_acc_sid,
            major_mlpid,
            j+1,
            beta,
            vr2,
            vp2,
            vt2,
            bin_edges,
            bin_cents,
        )
        samp_anisotropy_cb.append(_data)

        ### Osipkov-Merritt samples

        # Get the sample
        mask = (sample_data_om['z0_sid'] == z0_sid) &\
               (sample_data_om['major_acc_sid'] == major_acc_sid) &\
               (sample_data_om['major_mlpid'] == major_mlpid) &\
               (sample_data_om['merger_number'] == j+1)
        indx = np.where(mask)[0]
        assert len(indx) == 1, 'Something went wrong'
        indx = indx[0]

        sample = sample_data_om['sample'][indx]

        # Adaptive binning
        adaptive_binning_kwargs = {'rmin':np.max([np.min(sample.r().to_value(apu.kpc)), r_softening]),
                                   'n':np.min([500, len(sample)//10]),
                                   'rmax':np.max(sample.r().to_value(apu.kpc)),
                                   'bin_mode':'exact numbers',
                                   'bin_equal_n':True,
                                   'end_mode':'ignore',
                                   'bin_cents_mode':'median'}
        bin_edges, bin_cents, _ = pkin.get_radius_binning(sample, 
            **adaptive_binning_kwargs)
        bin_size = bin_edges[1:] - bin_edges[:-1]

        # Compute anisotropy profile
        compute_betas_kwargs = {'use_dispersions':True,
                               'return_kinematics':True}
        beta,vr2,vp2,vt2 = \
            pkin.compute_betas_bootstrap(sample, bin_edges, n_bootstrap=n_bs,
                compute_betas_kwargs=compute_betas_kwargs)

        # Store data
        _data = (
            z0_sid,
            major_acc_sid,
            major_mlpid,
            j+1,
            beta,
            vr2,
            vp2,
            vt2,
            bin_edges,
            bin_cents,
        )
        samp_anisotropy_om.append(_data)
        

# Save the data as a pickle
header = ['z0_sid','major_acc_sid','major_mlpid','merger_number',
          'sample_beta','sample_vr2','sample_vp2','sample_vt2','bin_edges',
          'bin_cents'
          ]
with open(os.path.join(analysis_dir,'sample_anisotropy_profile_cb.pkl'),'wb') as handle:
    pickle.dump([header,samp_anisotropy_cb], handle)
with open(os.path.join(analysis_dir,'sample_anisotropy_profile_om.pkl'),'wb') as handle:
    pickle.dump([header,samp_anisotropy_om], handle)

# Also save as a structured array
samp_anisotropy_dtype = np.dtype([
    ('z0_sid',int),
    ('major_acc_sid',int),
    ('major_mlpid',int),
    ('merger_number',int),
    ('beta',object),
    ('vr2',object),
    ('vp2',object),
    ('vt2',object),
    ('bin_edges',object),
    ('bin_cents',object)
    ])
samp_anisotropy_cb = np.array(samp_anisotropy_cb, dtype=samp_anisotropy_dtype)
samp_anisotropy_om = np.array(samp_anisotropy_om, dtype=samp_anisotropy_dtype)
np.save(os.path.join(analysis_dir,'sample_anisotropy_profile_cb.npy'), samp_anisotropy_cb)
np.save(os.path.join(analysis_dir,'sample_anisotropy_profile_om.npy'), samp_anisotropy_om)

### Load the stashed sample anisotropy profiles if necessary

In [ ]:
analysis_version = 'v1.1'
analysis_dir = os.path.join(mw_analog_dir,'analysis',analysis_version)

samp_anisotropy_cb = np.load(os.path.join(analysis_dir,
    'sample_anisotropy_profile_cb.npy'), allow_pickle=True)
samp_anisotropy_om = np.load(os.path.join(analysis_dir,
    'sample_anisotropy_profile_om.npy'), allow_pickle=True)

### Fit the anisotropies

In [ ]:
verbose=True

# Begin logging
log_filename = './log/8_assess_rotating_anisotropy_fits.log'
if os.path.exists(log_filename):
    os.remove(log_filename)
logging.basicConfig(filename=log_filename, level=logging.INFO, filemode='w', 
    force=True)
logging.info('Beginning fitting anisotropy models to samples: '+\
             time.strftime('%a, %d %b %Y %H:%M:%S',time.localtime()))

# Models to fit and fitting params
models = [pkin.beta_constant, pkin.beta_osipkov_merritt]
inits = [[0.], [10.]]
mcmc_labels = [[r'$\beta$',],
               [r'$r_a$',]]
param_names = [['beta'],
               ['ra']]
plot_labels = [r'Constant-$\beta$',
               r'Osipkov-Merritt']
plot_colors = ['DodgerBlue','Crimson']
plot_linestyles = ['solid','dashed']
plot_zorders = [1,2]
df_type = ['constant_beta','osipkov_merritt']
version = 'anisotropy_params_sample_fits'
fit_version = [version,version]

# MCMC params
nwalkers = 10
nit = 200
ncut = 100
nprocs = 8

anisotropy_params = np.zeros((len(samp_anisotropy_cb),2,3))

for i in range(n_mw):
    # if i > 0: continue

    # Get the primary
    primary = tree_primaries[i]
    z0_sid = primary.subfind_id[0]
    major_mergers = primary.tree_major_mergers
    n_major = primary.n_major_mergers

    for j in range(n_major):
        # if j > 0: continue
        if verbose: print(f'Calculating anisotropy profiles for MW analog '
                          f'{i+1}/{n_mw}, merger {j+1}/{n_major}', end='\r')

        # # Get the major merger
        major_merger = primary.tree_major_mergers[j]
        major_acc_sid = major_merger.subfind_id[0]
        major_mlpid = major_merger.secondary_mlpid
        r_softening = putil.get_softening_length('stars', z=0, physical=True)

        # Get the sample indx, should be same for cb and om
        mask = (samp_anisotropy_cb['z0_sid'] == z0_sid) &\
               (samp_anisotropy_cb['major_acc_sid'] == major_acc_sid) &\
               (samp_anisotropy_cb['major_mlpid'] == major_mlpid) &\
               (samp_anisotropy_cb['merger_number'] == j+1)
        indx = np.where(mask)[0]
        assert len(indx) == 1, 'Something went wrong'
        
        ### Fit the samples
        samples = [samp_anisotropy_cb,
                   samp_anisotropy_om]

        for k in range(2):
            if verbose:
                msg = f'Fitting anisotropy model'+df_type[k]
                logging.info(msg)
                print(msg)
            
            # Plotting directory
            this_fig_dir = os.path.join(fig_dir, fit_version[k], str(z0_sid), 
                'merger_'+str(j+1))
            os.makedirs(this_fig_dir, exist_ok=True)

            model = models[k]
            init = inits[k]

            bin_cents = np.median(samples[k]['bin_cents'][indx],axis=0)
            betas = samples[k]['beta'][indx][0]
            lbeta, mbeta, ubeta = np.percentile(betas, [16,50,84], axis=0)
            sbeta = ubeta-lbeta

            beta_mask = np.isfinite(mbeta) & (sbeta > 0.)
            bin_cents = bin_cents[beta_mask]
            mbeta = mbeta[beta_mask]
            lbeta = lbeta[beta_mask]
            ubeta = ubeta[beta_mask]
            sbeta = sbeta[beta_mask]

            # Optimizing
            opt_fn = lambda params: mloglike_beta(params, model, bin_cents, 
                mbeta, sigma=sbeta, mass=None, usr_log_prior=None, 
                usr_log_prior_params=[], parts=False)
            opt = scipy.optimize.minimize(opt_fn, init, method='Nelder-Mead',
                options={'maxiter':1000,})

            # MCMC
            def llfunc(params):
                return loglike_beta(params, model, bin_cents, mbeta, 
                    sigma=sbeta, mass=None, usr_log_prior=None, 
                    usr_log_prior_params=[], parts=False)
            mcmc_init = np.array([
                opt.x+0.05*np.random.randn(len(opt.x)) for i in range(nwalkers)
            ])
            with multiprocessing.Pool(processes=nprocs) as pool:
                sampler = emcee.EnsembleSampler(nwalkers, len(mcmc_init[0]), 
                    llfunc, pool=pool)
                sampler.run_mcmc(mcmc_init, nit, progress=True)
            chain = sampler.get_chain(flat=True, discard=ncut)

            # Corner plot for these results
            figc = corner.corner(chain, labels=mcmc_labels[k], 
                quantiles=[0.16,0.5,0.84], truths=opt.x, 
                truth_color='Red')
            figname = os.path.join(this_fig_dir,df_type[k]+'_corner.png')
            figc.savefig(figname)
            plt.close(figc)

            # Make a figure of the anisotropy and the fits
            fig = plt.figure()
            ax = fig.add_subplot(111)
            ax.plot(bin_cents, mbeta, color='Black')
            ax.fill_between(bin_cents, lbeta, ubeta, color='Black', alpha=0.3)
            nplot = 50
            plot_ind = np.random.choice(range(0,len(chain)), nplot, 
                replace=False)
            for l in range(nplot):
                param = chain[plot_ind[l]]
                y = model(bin_cents, *param)
                ax.plot(bin_cents, y, color='Red', alpha=0.1)
            ax.set_xlabel('Radius [kpc]')
            ax.set_ylabel(r'$\beta$')
            fig.savefig(os.path.join(this_fig_dir,df_type[k]+'_fit.png'))
            plt.close(fig)

            # Save the results
            anisotropy_params[indx,k,:] = np.percentile(chain[:,0], [16,50,84])



### Get merger data and compare

In [ ]:
analysis_version = 'v1.1'
analysis_dir = os.path.join(mw_analog_dir,'analysis',analysis_version)

# Load the merger information
merger_data = np.load(os.path.join(analysis_dir,'merger_data.npy'), 
    allow_pickle=True)

In [ ]:
fig = plt.figure(figsize=(6,10))
axs = fig.subplots(nrows=2, ncols=1)

beta_fit = merger_data['anisotropy']
beta_sample = anisotropy_params[:,0,1]

axs[0].scatter(beta_sample, beta_fit, color='DodgerBlue', alpha=0.5, zorder=2)
axs[0].plot([0,1],[0,1], color='Black', linestyle='dashed', zorder=1)
axs[0].set_xlabel(r'Sample $\beta$')
axs[0].set_ylabel(r'Fit $\beta$')

ra_fit = merger_data['ra']
ra_sample = anisotropy_params[:,1,1]
axs[1].scatter(ra_sample, ra_fit, color='DodgerBlue', alpha=0.5, zorder=2)
axs[1].plot([0,1000],[0,1000], color='Black', linestyle='dashed', zorder=1)
axs[1].set_xlabel(r'Sample $r_{a}$')
axs[1].set_ylabel(r'Fit $r_{a}$')
axs[1].set_xscale('log')
axs[1].set_yscale('log')

plt.show()

In [ ]:
def moving_median(x, w):
    # Compute the moving median, not average
    n = len(x)-w+1
    y = np.zeros(n)
    for i in range(n):
        y[i] = np.median(x[i:i+w])
    return y

In [ ]:
fig = plt.figure(figsize=(6,10))
axs = fig.subplots(nrows=2, ncols=1)

krot = np.abs( merger_data['krot'] )

# Delta beta
beta_fit = merger_data['anisotropy']
beta_sample = anisotropy_params[:,0,1]
delta_beta = (beta_sample-beta_fit)/beta_fit

# Plot delta beta
axs[0].scatter(np.abs(krot), delta_beta, color='DodgerBlue', alpha=0.5, zorder=2)
axs[0].set_xlabel(r'$k_{\mathrm{rot}}$')
axs[0].set_ylabel(r'$\Delta \beta$ [fractional]')
axs[0].set_ylim(-0.1,1.1)

# Moving median of delta beta and linear fit
krot_sort = np.argsort(krot)
krot_sorted = krot[krot_sort]
delta_beta_sorted = delta_beta[krot_sort]
krot_mm = moving_median(krot_sorted, 9)
delta_beta_mm = moving_median(delta_beta_sorted, 9)
axs[0].plot(krot_mm, delta_beta_mm, color='Black', zorder=1)
mock_x = np.linspace(0,1,2)
delta_beta_linfit = np.polyfit(krot, delta_beta, 1)
axs[0].plot(mock_x, delta_beta_linfit[0]*mock_x+delta_beta_linfit[1], 
    color='Red')

# Delta ra
ra_fit = merger_data['ra']
ra_sample = anisotropy_params[:,1,1]
delta_ra = (ra_sample-ra_fit)/ra_fit

# Plot delta ra
axs[1].scatter(krot, delta_ra, color='DodgerBlue', alpha=0.5, zorder=2)
axs[1].set_xlabel(r'$k_{\mathrm{rot}}$')
axs[1].set_ylabel(r'$\Delta r_{a}$ [fractional]')
axs[1].set_ylim(-0.4,0.1)

# Moving median of delta ra
delta_ra_sorted = delta_ra[krot_sort]
delta_ra_mm = moving_median(delta_ra_sorted, 9)
axs[1].plot(krot_mm, delta_ra_mm, color='Black', zorder=1)
delta_ra_linfit = np.polyfit(krot, delta_ra, 1)
axs[1].plot(mock_x, delta_ra_linfit[0]*mock_x+delta_ra_linfit[1], 
    color='Red')

plt.show()

In [ ]:
fig = plt.figure(figsize=(6,10))
axs = fig.subplots(nrows=2, ncols=1)

krot = np.abs( merger_data['krot'] )

# Delta beta
beta_fit = merger_data['anisotropy']
beta_sample = anisotropy_params[:,0,1]
delta_beta = (beta_sample-beta_fit)#/beta_fit

# Plot delta beta
axs[0].scatter(np.abs(krot), delta_beta, color='DodgerBlue', alpha=0.5, zorder=2)
axs[0].set_xlabel(r'$k_{\mathrm{rot}}$')
axs[0].set_ylabel(r'$\Delta \beta$ [fractional]')
axs[0].set_ylim(-0.05,0.25)

# Moving median of delta beta and linear fit
krot_sort = np.argsort(krot)
krot_sorted = krot[krot_sort]
delta_beta_sorted = delta_beta[krot_sort]
krot_mm = moving_median(krot_sorted, 9)
delta_beta_mm = moving_median(delta_beta_sorted, 9)
axs[0].plot(krot_mm, delta_beta_mm, color='Black', zorder=1)
mock_x = np.linspace(0,1,2)
delta_beta_linfit = np.polyfit(krot, delta_beta, 1)
axs[0].plot(mock_x, delta_beta_linfit[0]*mock_x+delta_beta_linfit[1], 
    color='Red')

# Delta ra
ra_fit = merger_data['ra']
ra_sample = anisotropy_params[:,1,1]
delta_ra = (ra_sample-ra_fit)#/ra_fit

# Plot delta ra
axs[1].scatter(krot, delta_ra, color='DodgerBlue', alpha=0.5, zorder=2)
axs[1].set_xlabel(r'$k_{\mathrm{rot}}$')
axs[1].set_ylabel(r'$\Delta r_{a}$ [kpc]')
axs[1].set_ylim(-10,5)

# Moving median of delta ra
delta_ra_sorted = delta_ra[krot_sort]
delta_ra_mm = moving_median(delta_ra_sorted, 9)
axs[1].plot(krot_mm, delta_ra_mm, color='Black', zorder=1)
delta_ra_linfit = np.polyfit(krot, delta_ra, 1)
axs[1].plot(mock_x, delta_ra_linfit[0]*mock_x+delta_ra_linfit[1], 
    color='Red')

plt.show()